In [1]:
import pandas as pd
import requests
from loguru import logger
import os

In [48]:
import requests
import pandas as pd
import re

battle_ref = "gen9ou-2437419661"  # change this for another battle
url = f"https://replay.pokemonshowdown.com/{battle_ref}.json"

# Download JSON
response = requests.get(url)
data = response.json()
log = data.get("log", "")

lines = log.splitlines()

def parse_log_with_block_entry_hp(lines, battle_ref):
    current_ts = None
    poke_p1 = None
    poke_p2 = None
    hp_p1 = 100
    hp_p2 = 100
    pre_hp_p1 = None
    pre_hp_p2 = None
    block_entry_hp_p1 = None
    block_entry_hp_p2 = None
    action_p1 = ""
    action_p2 = ""
    rows = []
    victory_flag = 0

    def append_row_for_ts(ts):
        nonlocal rows, poke_p1, poke_p2, hp_p1, hp_p2, pre_hp_p1, pre_hp_p2, block_entry_hp_p1, block_entry_hp_p2, action_p1, action_p2, victory_flag
        if not (poke_p1 and poke_p2):
            return
        ori1 = block_entry_hp_p1 if block_entry_hp_p1 is not None else (pre_hp_p1 if pre_hp_p1 is not None else hp_p1)
        ori2 = block_entry_hp_p2 if block_entry_hp_p2 is not None else (pre_hp_p2 if pre_hp_p2 is not None else hp_p2)
        diff = (hp_p1 - ori1) - (hp_p2 - ori2)
        rows.append({
            "Poke_battle_id": battle_ref,
            "Turn_id": ts,
            "Poke_p1": poke_p1,
            "Poke_p2": poke_p2,
            "PokemonP1_HP_ORI": int(ori1),
            "PokemonP2_HP_ORI": int(ori2),
            "Action_P1": action_p1,
            "Action_P2": action_p2,
            "PokemonP1_HP": int(hp_p1),
            "PokemonP2_HP": int(hp_p2),
            "Difference": int(diff),
            "Move": 0,
            "Victory_flag": victory_flag
        })
        victory_flag = 0

    for line in lines:
        if not line.startswith("|"):
            continue
        parts = line.split("|")
        if len(parts) < 2:
            continue
        evt = parts[1]

        
        if evt == "t:":
            new_ts = parts[2] if len(parts) > 2 else None
            if current_ts is not None:
                append_row_for_ts(current_ts)
            current_ts = new_ts
            pre_hp_p1 = hp_p1
            pre_hp_p2 = hp_p2
            block_entry_hp_p1 = None
            block_entry_hp_p2 = None
            action_p1 = ""
            action_p2 = ""
            continue

        if evt == "switch":
            player = parts[2]
            pokemon = parts[3].split(",")[0] if len(parts) > 3 else None
            hp_str = parts[4] if len(parts) > 4 else None
            if player.startswith("p1"):
                poke_p1 = pokemon
                if hp_str and "/" in hp_str:
                    try: hp_p1 = int(hp_str.split("/")[0])
                    except: pass
                if block_entry_hp_p1 is None:
                    block_entry_hp_p1 = hp_p1
                action_p1 = "Switch"
            elif player.startswith("p2"):
                poke_p2 = pokemon
                if hp_str and "/" in hp_str:
                    try: hp_p2 = int(hp_str.split("/")[0])
                    except: pass
                if block_entry_hp_p2 is None:
                    block_entry_hp_p2 = hp_p2
                action_p2 = "Switch"
            continue

        if evt == "drag":
            player = parts[2]
            pokemon = parts[3].split(",")[0] if len(parts) > 3 else None
            hp_str = parts[4] if len(parts) > 4 else None
            if player.startswith("p1"):
                poke_p1 = pokemon
                if hp_str and "/" in hp_str:
                    try: hp_p1 = int(hp_str.split("/")[0])
                    except: pass
                if block_entry_hp_p1 is None:
                    block_entry_hp_p1 = hp_p1
            elif player.startswith("p2"):
                poke_p2 = pokemon
                if hp_str and "/" in hp_str:
                    try: hp_p2 = int(hp_str.split("/")[0])
                    except: pass
                if block_entry_hp_p2 is None:
                    block_entry_hp_p2 = hp_p2
            continue

        if evt == "move":
            player = parts[2]
            move = parts[3] if len(parts) > 3 else ""
            if player.startswith("p1"): action_p1 = move
            elif player.startswith("p2"): action_p2 = move
            continue

        if evt == "cant":
            player = parts[2]
            if player.startswith("p1"): action_p1 = "cant"
            elif player.startswith("p2"): action_p2 = "cant"
            continue

        if evt == "faint":
            player = parts[2]
            if player.startswith("p1"):
                victory_flag = -1
            elif player.startswith("p2"):
                victory_flag = 1
            continue

        if evt in ("-damage", "-heal"):
            player = parts[2]
            hp_token = parts[3] if len(parts) > 3 else ""
            match = re.search(r"(\d+)(?:/(\d+))?", hp_token)
            if match:
                hp_val = int(match.group(1))
                if player.startswith("p1"): hp_p1 = hp_val
                elif player.startswith("p2"): hp_p2 = hp_val
            continue

    if current_ts is not None:
        append_row_for_ts(current_ts)

    return rows

rows = parse_log_with_block_entry_hp(lines, battle_ref)
df = pd.DataFrame(rows)

# Compute Move
for i in range(len(df) - 1):
    cur, nxt = df.loc[i], df.loc[i+1]
    mv = 0
    if str(cur["Action_P1"]) in ("Switch",""): mv += 1
    if str(cur["Action_P2"]) in ("Switch",""): mv -= 1
    if str(nxt["Action_P1"]) in ("Switch",""): mv -= 1
    if str(nxt["Action_P2"]) in ("Switch",""): mv += 1
    df.at[i,"Move"] = mv

# Save CSV locally
out_path = f"{battle_ref}.csv"
df.to_csv(out_path, index=False)
print(f"Saved to {out_path} with {len(df)} rows")
df.head(20)


Saved to gen9ou-2437419661.csv with 20 rows


,Poke_battle_id,Turn_id,Poke_p1,Poke_p2,PokemonP1_HP_ORI,PokemonP2_HP_ORI,Action_P1,Action_P2,PokemonP1_HP,PokemonP2_HP,Difference,Move,Victory_flag
0,gen9ou-2437419661,1757320834,Ting-Lu,Mew,100,100,Switch,Switch,100,100,0,0,0
1,gen9ou-2437419661,1757320839,Ting-Lu,Mew,100,100,cant,Imprison,100,100,0,-1,0
2,gen9ou-2437419661,1757320848,Meowscarada,Mew,100,100,Switch,Stealth Rock,100,100,0,1,0
3,gen9ou-2437419661,1757320853,Meowscarada,Mew,100,100,Knock Off,Transform,91,62,29,0,0
4,gen9ou-2437419661,1757320871,Meowscarada,Mew,91,62,Struggle,Knock Off,24,37,-42,-1,0
5,gen9ou-2437419661,1757320902,Alomomola,Mew,100,37,Switch,Triple Axel,83,3,17,2,0
6,gen9ou-2437419661,1757320919,Alomomola,Necrozma,83,100,Flip Turn,Switch,83,90,10,-1,0
7,gen9ou-2437419661,1757320925,Ting-Lu,Necrozma,100,90,Switch,,100,96,-6,1,0
8,gen9ou-2437419661,1757320937,Ting-Lu,Mew,100,3,Stealth Rock,Switch,100,3,0,-1,0
9,gen9ou-2437419661,1757320942,Ting-Lu,Slowking-Galar,100,100,Whirlwind,Spikes,100,88,12,0,0
